In [5]:
import numpy as np

This document serves to explain how the minimum distances are calculated by working through an example of a cube in a periodic box.  

## Set up
#### Defining Our System
First, we need to define our system. For this example, we will have a single cube of position $\mathbf{s_{position}}$ inside a cubic periodic box of size 10x10x10. 
$$\mathbf{s_{position}} = \begin{bmatrix} 3&3&3 \end{bmatrix}$$

<center><img src="Image/set_up.png" alt="set_up" width="800"/> </center>

#### Defining Our Shape
We can define our shape by defining its vertices. These vertices are user defined, and describe the type of shape that we are working with. When dealing with a system of multiple particles of the same shape, the vertices only need to be defined for one. Thus, the origin of the defined vertices usually does not align with the origin of the periodic box. In this example, the vertices are defined with the origin as the center of the cube, but it is not a requirement to have the origin as the center of the shape when defining the vertices.
<center><img src="Image/zoom_in_set_up.png" alt="zoom_in_set_up" width="800"/></center>

We will define the vertices of the cube as:
$$\mathbf{V} = \begin{bmatrix} \mathbf{v_1}\\ \mathbf{v_2}\\ \mathbf{v_3}\\ \mathbf{v_4}\\ \mathbf{v_5}\\ \mathbf{v_6}\\ \mathbf{v_7}\\ \mathbf{v_8} \end{bmatrix} = \begin{bmatrix*}[r] 1&1&1\\ -1&1&1\\ 1&-1&1\\ 1&1&-1\\ -1&-1&1\\ -1&1&-1\\ 1&-1&-1\\ -1&-1&-1 \end{bmatrix*}$$

From the vertices, we can get the edge vectors, which are taken to be the difference between the two corresponding vertices for each edge. We define two variables: edge-vertex neighbors ($\mathbf{N_{vert}}$) and edges ($\mathbf{E}$), where $\mathbf{N_{vert}}$ contains the corresponding vertices for each edge, and $\mathbf{E}$ contains the difference between the vertices in the two columns of $\mathbf{N_{vert}}$:
$$\mathbf{N_{vert}} = \begin{bmatrix} \mathbf{v_1}&\mathbf{v_2}\\ \mathbf{v_1}&\mathbf{v_3}\\ \mathbf{v_1}&\mathbf{v_4}\\ \mathbf{v_2}&\mathbf{v_5}\\ \mathbf{v_2}&\mathbf{v_6}\\ \mathbf{v_3}&\mathbf{v_5}\\ \mathbf{v_3}&\mathbf{v_7}\\ \mathbf{v_4}&\mathbf{v_6}\\ \mathbf{v_4}&\mathbf{v_7}\\ \mathbf{v_5}&\mathbf{v_8}\\ \mathbf{v_6}&\mathbf{v_8}\\ \mathbf{v_7}&\mathbf{v_8} \end{bmatrix}$$

(Note: In the code, the edge-vertex neighbors variable contains only integer values, which correspond to the indices of the vertices)

$$\mathbf{E} = \begin{bmatrix} \mathbf{e_1}\\ \mathbf{e_2}\\ \mathbf{e_3}\\ \mathbf{e_4}\\ \mathbf{e_5}\\ \mathbf{e_6}\\ \mathbf{e_7}\\ \mathbf{e_8}\\ \mathbf{e_9}\\ \mathbf{e_{10}}\\ \mathbf{e_{11}}\\ \mathbf{e_{12}} \end{bmatrix} = \begin{bmatrix} \mathbf{v_2}-\mathbf{v_1}\\ \mathbf{v_3}-\mathbf{v_1}\\ \mathbf{v_4}-\mathbf{v_1}\\ \mathbf{v_5}-\mathbf{v_2}\\ \mathbf{v_6}-\mathbf{v_2}\\ \mathbf{v_5}-\mathbf{v_3}\\ \mathbf{v_7}-\mathbf{v_3}\\ \mathbf{v_6}-\mathbf{v_4}\\ \mathbf{v_7}-\mathbf{v_4}\\ \mathbf{v_8}-\mathbf{v_5}\\ \mathbf{v_8}-\mathbf{v_6}\\ \mathbf{v_8}-\mathbf{v_7} \end{bmatrix} = \begin{bmatrix*}[r] -2&0&0\\ 0&-2&0\\ 0&0&-2\\ 0&-2&0\\ 0&0&-2\\ -2&0&0\\ 0&0&-2\\ -2&0&0\\ 0&-2&0\\ 0&0&-2\\ 0&-2&0\\ -2&0&0 \end{bmatrix*}$$

For a cube, the normals of the faces are calculated by taking the cross product of two adjacent edges. In the code, the software package, Coxeter, is used to calculate the normals. For this example, the normals of the faces are:
$$\mathbf{F} = \begin{bmatrix} \mathbf{f_1}\\ \mathbf{f_2}\\ \mathbf{f_3}\\ \mathbf{f_4}\\ \mathbf{f_5}\\ \mathbf{f_6} \end{bmatrix} = \begin{bmatrix} \mathbf{e_1} \times \mathbf{e_2}\\ \mathbf{e_3} \times \mathbf{e_1}\\ \mathbf{e_2} \times \mathbf{e_3} \\ \mathbf{e_5} \times \mathbf{e_4} \\ \mathbf{e_6} \times \mathbf{e_7} \\ \mathbf{e_9} \times \mathbf{e_8} \end{bmatrix} = \begin{bmatrix*}[r] 0&0&4\\ 0&4&0\\ 4&0&0\\ -4&0&0\\ 0&-4&0\\ 0&0&-4 \end{bmatrix*}$$

We also define another variable, edge-face neighbors ($\mathbf{M_{face}}$), which contains the corresponding faces for each edge. If an edge vector is oriented such that it is pointing upwards, the face on its left corresponds with the first column and the face on its right corresponds with the second column of the respective edge in edge-face neighbors.
$$\mathbf{M_{face}}  = \begin{bmatrix} \mathbf{f_1}&\mathbf{f_2}\\ \mathbf{f_3}&\mathbf{f_1}\\ \mathbf{f_2}&\mathbf{f_3}\\ \mathbf{f_1}&\mathbf{f_4}\\ \mathbf{f_4}&\mathbf{f_2}\\ \mathbf{f_5}&\mathbf{f_1}\\ \mathbf{f_3}&\mathbf{f_5}\\ \mathbf{f_2}&\mathbf{f_6}\\ \mathbf{f_6}&\mathbf{f_3}\\ \mathbf{f_5}&\mathbf{f_4}\\ \mathbf{f_4}&\mathbf{f_6}\\ \mathbf{f_6}&\mathbf{f_5} \end{bmatrix}$$

(Note: In the code, the edge-face neighbors variable contains only integer values, which correspond to the indices of the faces)

In [ ]:
#In the code, it looks something like this (variables are named the same as they are in the code)


## What is a Zone?
We define a zone such that these two variables, $\mathbf{A}$ and $\mathbf{a}$, satisfy this condition for all grid points, $\mathbf{x}$, located within the zone:
$$\mathbf{A} \cdot \mathbf{x} \leq \mathbf{a}$$

The matrix $\mathbf{A}$ contains the normals (pointing outward from the zone) of all the planar boundaries associated with the zone, and the vector $\mathbf{a}$ contains the bounds.

(Note: Matrix $\mathbf{A}$ is not normalized in the code because the bounds $\mathbf{a}$ are calculated by taking the dot product between matrix $\mathbf{A}$ and the relevent points ($\mathbf{a} = \mathbf{A} \cdot \mathbf{y_{point}} = | \text{A}| \mathbf{A_{unit \ vector}} \cdot \mathbf{y_{point}}$). Because of this, the magnitudes of the normals in matrix $\mathbf{A}$ cancel out with bounds $\mathbf{a}$.)

Knowing what zone a grid point is in let's us know how the minimum distance should be calulated. For grid points inside a vertex zone, the distance is calculated as the distance between two points, the grid point and the vertex. For grid points inside an edge zone, the distance is calculated as the distance between a point (the grid point) and a line (the edge). For grid points inside a face zone, the distance is calculated as the distance between a point (the grid point) and a plane (the face). If a grid point is located on the boundary between two differenct zones, it does not matter which zone, and therefore which distance calculation, is used since the distance calculated will be the same regardless. The minimum distance is continuous across boundaries. 

## Vertex Zone Example

#### Creating the Zone
We will now work through how a vertex zone is created and how the minimum distance calculations are done for the grid points that exist within this vertex zone.

For this example, we will look at the zone created by vertex $\mathbf{v_4}$. First, we will calculate the corresponding $\mathbf{A}$ matrix, which we will call $\mathbf{P}$. The values for matrix $\mathbf{P}$ are found to be the corresponding edges to vertex $\mathbf{v_4}$ in the direction pointing away from the vertex. By looking at $\mathbf{N_{vert}}$, we see that edges $\mathbf{e_3}$, $\mathbf{e_8}$, and $\mathbf{e_9}$ correspond with vertex $\mathbf{v_4}$. For $\mathbf{e_3}$, we see that $\mathbf{v_4}$ is in the second column of $\mathbf{N_{vert}}$, which tells us that the vector $\mathbf{e_3}$ is pointing towards $\mathbf{v_4}$, so we need to take the negative of vector $\mathbf{e_3}$ when calculating $\mathbf{P}$. For $\mathbf{e_8}$ and $\mathbf{e_9}$, $\mathbf{v_4}$ is in the first column of $\mathbf{N_{vert}}$, which tells us that the vectors $\mathbf{e_8}$ and $\mathbf{e_9}$ are pointing away from $\mathbf{v_4}$. Therefore, $\mathbf{P}$ is found to be:

$$\mathbf{P} = \begin{bmatrix*}[r] -\mathbf{e_3}\\ \mathbf{e_8}\\ \mathbf{e_9} \end{bmatrix*} = \begin{bmatrix*}[r] 0&0&2\\ -2&0&0\\ 0&-2&0 \end{bmatrix*}$$

Now we can calculate the corresponding $\mathbf{a}$ vector, which we will call $\mathbf{p}$. The values in $\mathbf{p}$ are calculated by taking the dot product between each plane in $\mathbf{P}$ with a point that lies on the respective plane. For vertex zones, the corresponding vertex lies in all the planes in $\mathbf{P}$. We have to keep in mind that the origin of our periodic box is different from the origin we set when defining our vertices, so we have to add the position $\mathbf{s_{position}}$ of the cube to our vertex. For our example, $\mathbf{p}$ is found to be:

$$\mathbf{p} = \mathbf{P} \cdot \begin{pmatrix} \mathbf{v_4} + \mathbf{s_{position}} \end{pmatrix} = \begin{bmatrix*}[r] 0&0&2\\ -2&0&0\\ 0&-2&0 \end{bmatrix*} \begin{bmatrix*}[r] 4\\ 4\\ 2 \end{bmatrix*} = \begin{bmatrix*}[r] 4\\ -8\\ -8 \end{bmatrix*}$$

<center><img src="Image/vertex_zone.png" alt="vertex_zone" width="800"/></center>

In [12]:
#How it would look in the code:

#### Validating the Zone
Our vertex zone is all set up! Now we can test out some grid points to make sure the condition $\mathbf{P} \cdot \mathbf{x} \leq \mathbf{p}$ holds true for points inside the zone, but is false for points outside the zone. 

<ins>__Outside Points:__</ins>

Let's try these points, which should be __outside__ the zone: 
$$\mathbf{x_1} = \begin{bmatrix} 5&0&0 \end{bmatrix}, \; \; \mathbf{x_2} = \begin{bmatrix} 5&2&5 \end{bmatrix}, \; \; \mathbf{x_3} = \begin{bmatrix} 3&3&5 \end{bmatrix}$$

$$\mathbf{P} \cdot \mathbf{x_1} = \begin{bmatrix*}[r] 0&0&2\\ -2&0&0\\ 0&-2&0 \end{bmatrix*} \begin{bmatrix}5\\ 0\\ 0 \end{bmatrix} = \begin{bmatrix} 0\\ -10\\ 0 \end{bmatrix} \nleq \begin{bmatrix*}[r] 4\\ -8\\ -8 \end{bmatrix*} = \mathbf{p}$$

Although 0 < 4 in the first row and -10 < -8 in the second row, 0 > -8 in the third row, so $\mathbf{P} \cdot \mathbf{x_1}$ is not less than or equal to $\mathbf{p}$. Therefore, point $\mathbf{x_1}$ is __outside__ this vertex zone, which is what we expected.

$$\mathbf{P} \cdot \mathbf{x_2} = \begin{bmatrix*}[r] 0&0&2\\ -2&0&0\\ 0&-2&0 \end{bmatrix*} \begin{bmatrix} 5\\ 2\\ 5 \end{bmatrix} = \begin{bmatrix*}[r] 10\\ -10\\ -4 \end{bmatrix*} \nleq \begin{bmatrix*}[r] 4\\ -8\\ -8 \end{bmatrix*} = \mathbf{p}$$

As can be seen, 10 > 4 and -4 > -8. Therefore, $\mathbf{P} \cdot \mathbf{x_2}$ is not less than or equal to $\mathbf{p}$, so point $\mathbf{x_2}$ is __outside__ this vertex zone.

$$\mathbf{P} \cdot \mathbf{x_3} = \begin{bmatrix*}[r] 0&0&2\\ -2&0&0\\ 0&-2&0 \end{bmatrix*} \begin{bmatrix} 3\\ 3\\ 5 \end{bmatrix} = \begin{bmatrix*}[r] 10\\ -6\\ -6 \end{bmatrix*} \nleq \begin{bmatrix*}[r] 4\\ -8\\ -8 \end{bmatrix*} = \mathbf{p}$$

Here, all the values of $\mathbf{P} \cdot \mathbf{x_3}$ is greater than $\mathbf{p}$, so point $\mathbf{x_3}$ is __outside__ this vertex zone.

<br></br>
<ins>__Inside and Boundary Points:__</ins>

Now let's try for a point __inside__ the zone. Let's look at point:
$$\mathbf{x_4} = \begin{bmatrix} 5&5&1 \end{bmatrix}$$

$$\mathbf{P} \cdot \mathbf{x_4} = \begin{bmatrix*}[r] 0&0&2\\ -2&0&0\\ 0&-2&0 \end{bmatrix*} \begin{bmatrix} 5\\ 5\\ 1 \end{bmatrix} = \begin{bmatrix*}[r] 2\\ -10\\ -10 \end{bmatrix*} \leq \begin{bmatrix*}[r] 4\\ -8\\ -8 \end{bmatrix*} = \mathbf{p}$$

All the values in $\mathbf{P} \cdot \mathbf{x_4}$ are less than the values in $\mathbf{p}$, so point $\mathbf{x_4}$ is __inside__ this vertex zone.


What about a point located on the boundary? It should also hold true for the condition, so let's look at point:
$$\mathbf{x_5} = \begin{bmatrix} 5&4&2 \end{bmatrix}$$

$$\mathbf{P} \cdot \mathbf{x_5} = \begin{bmatrix*}[r] 0&0&2\\ -2&0&0\\ 0&-2&0 \end{bmatrix*} \begin{bmatrix} 5\\ 4\\ 2 \end{bmatrix} = \begin{bmatrix*}[r] 4\\ -10\\ -8 \end{bmatrix*} \leq \begin{bmatrix*}[r] 4\\ -8\\ -8 \end{bmatrix*} = \mathbf{p}$$

All the values in $\mathbf{P} \cdot \mathbf{x_5}$ are less than or equal to $\mathbf{p}$, so point $\mathbf{x_5}$ is treated as being __inside__ this vertex zone.


In [1]:
#How it would look in code:

#### Distance Calculation

Since points $\mathbf{x_4}$ and $\mathbf{x_5}$ are inside our vertex zone, let's solve for their distances to the cube. For points located within a vertex zone, their distances to the shape is calculated by finding the distance between the points and the associated vertex. For this vertex zone, the position of the associated vertex is:

$$\mathbf{x_v} = \mathbf{v_4} + \mathbf{s_{position}} = \begin{bmatrix} 4&4&2 \end{bmatrix}$$

The distance between two points is calculated by:

$$d_{point-point} = \begin{vmatrix} \mathbf{point} - \mathbf{vertex} \end{vmatrix}$$

The distance between point $\mathbf{x_4}$ and the cube is found to be:

$$d_{x4} = \begin{vmatrix} \mathbf{x_4} - \mathbf{x_v} \end{vmatrix} = \begin{vmatrix} \begin{bmatrix} 5&5&1 \end{bmatrix} - \begin{bmatrix} 4&4&2 \end{bmatrix} \end{vmatrix} = \begin{vmatrix} \begin{bmatrix*}[r] 1&1&-1 \end{bmatrix*} \end{vmatrix} = \sqrt{1 + 1 + 1} = \sqrt{3}$$

The distance between point $\mathbf{x_5}$ and the cube is found to be:

$$d_{x5} = \begin{vmatrix} \mathbf{x_5} - \mathbf{x_v} \end{vmatrix} = \begin{vmatrix} \begin{bmatrix} 5&4&2 \end{bmatrix} - \begin{bmatrix} 4&4&2 \end{bmatrix} \end{vmatrix} = \begin{vmatrix} \begin{bmatrix*}[r] 1&0&0 \end{bmatrix*} \end{vmatrix} = \sqrt{1 + 0 + 0} = 1$$

In [2]:
#How it would look in code:

## Edge Zone Example

#### Creating the Zone

Let's now look at an example of an edge zone for this shape. We will look at the zone created by edge $\mathbf{e_1}$, and we will call the corresponding $\mathbf{A}$ matrix and $\mathbf{a}$ bounds for this zone $\mathbf{Q}$ and $\mathbf{q}$ respectively. Let's first calculate the constraint matrix $\mathbf{Q}$. We assume that every edge zone, regardless of shape, has only four planar boundaries. Two of these planar boundaries are determined from the edge vector, and the other two are determined by taking the cross product between the edge vector and the normals of its two neighboring faces. The normals of the two planar boundaries determined from the edge vector are calculated as the edge vector and the negative of the edge vector. By looking at $\mathbf{M_{face}}$, we see that face $\mathbf{f_1}$ is to the left of $\mathbf{e_1}$ and face $\mathbf{f_2}$ is to the right of $\mathbf{e_1}$. In order to make all the normals of the planar boundaries pointing outwards, the cross product between the edge vector $\mathbf{e_1}$ and its left neighboring face $\mathbf{f_1}$ needs to be multiplied by a negative one. The constraint matrix $\mathbf{Q}$ is found to be:

$$\mathbf{Q} = \begin{bmatrix*}[r] \mathbf{e_1} \\ - \mathbf{e_1} \\ \mathbf{e_1} \times \mathbf{f_2} \\ - \mathbf{e_1} \times \mathbf{f_1} \end{bmatrix*} = \begin{bmatrix*}[r] \begin{bmatrix*}[r]-2&0&0 \end{bmatrix*} \\ - \begin{bmatrix*}[r]-2&0&0 \end{bmatrix*} \\ \begin{bmatrix*}[r]-2&0&0 \end{bmatrix*} \times [0&4&0] \\ - \begin{bmatrix*}[r]-2&0&0 \end{bmatrix*} \times [0&0&4] \end{bmatrix*} = \begin{bmatrix*}[r] -2&0&0 \\ 2&0&0 \\ 0&0&-8 \\ 0&-8&0 \end{bmatrix*} $$

Now that we have calculated $\mathbf{Q}$, we can solve for $\mathbf{q}$ bounds. To solve for $\mathbf{q}$, we need to take the dot product between the normals in $\mathbf{Q}$ and points that lie on the planes. Unlike vertex zones, for an edge zone we need more than one point to solve for $\mathbf{q}$. The two points we will use will be the corresponding vertices to the edge vector. For this edge zone example, we find that the corresponding vertices are $\mathbf{v_1}$ and $\mathbf{v_2}$ by looking at the edge-vertex neighbors $\mathbf{N_{vert}}$. We need to make sure to add the position of the cube, $\mathbf{s_{position}}$ to our vertices before taking their dot products with their respective planar boundaries. Let's split up $\mathbf{Q}$ into its planar boundary components:

$$\mathbf{Q} = \begin{bmatrix} \mathbf{Q_1} \\ \mathbf{Q_2} \\ \mathbf{Q_3} \\ \mathbf{Q_{4}} \end{bmatrix} = \begin{bmatrix*}[r] -2&0&0 \\ 2&0&0 \\ 0&0&-8 \\ 0&-8&0 \end{bmatrix*} $$

By looking at $\mathbf{N_{vert}}$, we know that $\mathbf{v_1}$ is in the first collumn, which means it lies on the planar boundary created by $\mathbf{Q_2}$, and $\mathbf{v_2}$ is in the second collumn, which means it lies on the planar boundary created by $\mathbf{Q_1}$. Both vertices lie on the planar boundaries created by $\mathbf{Q_3}$ and $\mathbf{Q_4}$, so it does not matter which vertex we use when calculating $\mathbf{q}$. For this example, $\mathbf{q}$ is found to be:

$$\mathbf{q} = \begin{bmatrix} \mathbf{Q_1} \cdot \begin{pmatrix} \mathbf{v_2} + \mathbf{s_{position}} \end{pmatrix} \\ \mathbf{Q_2} \cdot \begin{pmatrix} \mathbf{v_1} + \mathbf{s_{position}} \end{pmatrix} \\ \mathbf{Q_3} \cdot \begin{pmatrix} \mathbf{v_1} + \mathbf{s_{position}} \end{pmatrix} \\ \mathbf{Q_4} \cdot \begin{pmatrix} \mathbf{v_1} + \mathbf{s_{position}} \end{pmatrix} \end{bmatrix} = \begin{bmatrix*}[r] [-2&0&0] \cdot [2&4&4] \\ [2&0&0] \cdot [4&4&4] \\ [0&0&-8] \cdot [4&4&4] \\ [0&-8&0] \cdot [4&4&4] \end{bmatrix*} = \begin{bmatrix*}[r] -4 \\ 8 \\ -32 \\ -32 \end{bmatrix*} $$

<center><img src="Image/edge_zone.png" alt="edge_zone" width="800"/></center>

In [3]:
#How it would look in code

#### Validating the Zone

Now let's check the edge zone with some points to verfiy that it satisfies the condition $\mathbf{Q} \cdot \mathbf{x} \leq \mathbf{q}$ for all grid points $\mathbf{x}$ that are within the edge zone. 

<ins>__Outside Points:__</ins>

Let's look at $\mathbf{x_2}$ and $\mathbf{x_3}$ from the vertex zone example. These two grid points should also be __outside__ for this example edge zone.

$$\mathbf{Q} \cdot \mathbf{x_2} = \begin{bmatrix*}[r] -2&0&0 \\ 2&0&0 \\ 0&0&-8 \\ 0&-8&0 \end{bmatrix*} \begin{bmatrix} 5 \\ 2 \\ 5 \end{bmatrix} = \begin{bmatrix*}[r] -10 \\ 10 \\ -40 \\ -16 \end{bmatrix*} \nleq \begin{bmatrix*}[r] -4 \\ 8 \\ -32 \\ -32 \end{bmatrix*}$$

Not every value in $\mathbf{Q} \cdot \mathbf{x_2}$ is less than their respective values in $\mathbf{q}$, so $\mathbf{x_2}$ is __outside__ this edge zone.

$$\mathbf{Q} \cdot \mathbf{x_3} = \begin{bmatrix*}[r] -2&0&0 \\ 2&0&0 \\ 0&0&-8 \\ 0&-8&0 \end{bmatrix*} \begin{bmatrix} 3 \\ 3 \\ 5 \end{bmatrix} = \begin{bmatrix*}[r] -6 \\ 6 \\ -40 \\ -24 \end{bmatrix*} \nleq \begin{bmatrix*}[r] -4 \\ 8 \\ -32 \\ -32 \end{bmatrix*}$$

There is one value in $\mathbf{Q} \cdot \mathbf{x_3}$ that is greater than $\mathbf{q}$, so point $\mathbf{x_3}$ is __outside__ this edge zone.

<br></br>
<ins>__Inside and Boundary Points:__</ins>

For a point located inside this zone, let's look at point $\mathbf{x_6}$, and for a point on the boundary, let's look at point $\mathbf{x_7}$:

$$\mathbf{x_6} = \begin{bmatrix} 3&5&5 \end{bmatrix}, \; \; \mathbf{x_7} = \begin{bmatrix} 3&4&5 \end{bmatrix} $$

$$\mathbf{Q} \cdot \mathbf{x_6} = \begin{bmatrix*}[r] -2&0&0 \\ 2&0&0 \\ 0&0&-8 \\ 0&-8&0 \end{bmatrix*} \begin{bmatrix} 3 \\ 5 \\ 5 \end{bmatrix} = \begin{bmatrix*}[r] -6 \\ 6 \\ -40 \\ -40 \end{bmatrix*} \leq \begin{bmatrix*}[r] -4 \\ 8 \\ -32 \\ -32 \end{bmatrix*}$$

All the values in $\mathbf{Q} \cdot \mathbf{x_6}$ are less than their respective values in $\mathbf{q}$, so point $\mathbf{x_6}$ is __inside__ this edge zone.

$$\mathbf{Q} \cdot \mathbf{x_7} = \begin{bmatrix*}[r] -2&0&0 \\ 2&0&0 \\ 0&0&-8 \\ 0&-8&0 \end{bmatrix*} \begin{bmatrix} 3 \\ 4 \\ 5 \end{bmatrix} = \begin{bmatrix*}[r] -6 \\ 6 \\ -40 \\ -32 \end{bmatrix*} \leq \begin{bmatrix*}[r] -4 \\ 8 \\ -32 \\ -32 \end{bmatrix*}$$

All the values in $\mathbf{Q} \cdot \mathbf{x_7}$ are less than or equal to their respective values in $\mathbf{q}$, so point $\mathbf{x_7}$, which lies on one of the planar boundaries, is seen as __inside__ this edge zone.

In [4]:
#How it would look in code

#### Distance Calculation

Let's solve for the distances from points $\mathbf{x_6}$ and $\mathbf{x_7}$ to the cube. For points located inside an edge zone, the distance calculated is the distance between a point and an infinite line. The point is our grid point, and the infinite line is described by the associated edge vector for the edge zone. To do this calculation, we need a point located on our line (edge vector), which we will take to be one of the associated vertices. For this example, we will define the associated vertex as:

$$\mathbf{x_e} = \mathbf{v_1} + \mathbf{s_{position}} = \begin{bmatrix} 4&4&4 \end{bmatrix}$$

We will define the unit vector of $\mathbf{e_1}$ as:

$$\mathbf{e_{1 \ unit}} = \frac{\mathbf{e_1}} {\begin{vmatrix} \textrm{e1} \end{vmatrix}} = \frac{\begin{bmatrix} -2&0&0 \end{bmatrix}}{2} = \begin{bmatrix} -1&0&0 \end{bmatrix} $$

We will define the distance between a point and a line as:

$$d_{point-line} = \begin{vmatrix} \begin{pmatrix} \mathbf{vertex} - \mathbf{point} \end{pmatrix} - \begin{Bmatrix} \begin{pmatrix} \mathbf{vertex} - \mathbf{point} \end{pmatrix} \cdot \mathbf{edge_{unit \ vector}} \end{Bmatrix} \mathbf{edge_{unit \ vector}} \end{vmatrix} $$

The distance between point $\mathbf{x_6}$ is found to be:

$$\begin{split}
d_{x6} & = \begin{vmatrix} \begin{pmatrix} \mathbf{x_e} - \mathbf{x_6} \end{pmatrix} - \begin{Bmatrix} \begin{pmatrix} \mathbf{x_e} - \mathbf{x_6} \end{pmatrix} \cdot \mathbf{e_{1 \ unit}} \end{Bmatrix} \mathbf{e_{1 \ unit}} \end{vmatrix} \\ 
 & = \begin{vmatrix} \begin{pmatrix} \begin{bmatrix} 4&4&4 \end{bmatrix} - \begin{bmatrix} 3&5&5 \end{bmatrix} \end{pmatrix} - \begin{Bmatrix} \begin{pmatrix} \begin{bmatrix} 4&4&4 \end{bmatrix} - \begin{bmatrix} 3&5&5 \end{bmatrix} \end{pmatrix} \cdot \begin{bmatrix} -1&0&0 \end{bmatrix} \end{Bmatrix} \begin{bmatrix} -1&0&0 \end{bmatrix} \end{vmatrix} \\ 
 & = \begin{vmatrix} \begin{bmatrix} 1&-1&-1 \end{bmatrix} - \begin{Bmatrix} \begin{bmatrix} 1&-1&-1 \end{bmatrix} \cdot \begin{bmatrix} -1&0&0 \end{bmatrix} \end{Bmatrix} \begin{bmatrix} -1&0&0 \end{bmatrix} \end{vmatrix} \\ 
 & = \begin{vmatrix} \begin{bmatrix} 1&-1&-1 \end{bmatrix} - \begin{Bmatrix} -1 \end{Bmatrix} \begin{bmatrix} -1&0&0 \end{bmatrix} \end{vmatrix} \\ 
 & = \begin{vmatrix} \begin{bmatrix} 1&-1&-1 \end{bmatrix} - \begin{bmatrix} 1&0&0 \end{bmatrix} \end{vmatrix} \\ 
 & = \begin{vmatrix} \begin{bmatrix} 0&-1&-1 \end{bmatrix} \end{vmatrix} \\ 
 & = \sqrt{0 + 1 + 1} \\ 
 & = \sqrt{2} 
\end{split}$$

The distance between point $\mathbf{x_7}$ is found to be:

$$\begin{split}
d_{x7} & = \begin{vmatrix} \begin{pmatrix} \mathbf{x_e} - \mathbf{x_7} \end{pmatrix} - \begin{Bmatrix} \begin{pmatrix} \mathbf{x_e} - \mathbf{x_7} \end{pmatrix} \cdot \mathbf{e_{1 \ unit}} \end{Bmatrix} \mathbf{e_{1 \ unit}} \end{vmatrix} \\ 
 & = \begin{vmatrix} \begin{pmatrix} \begin{bmatrix} 4&4&4 \end{bmatrix} - \begin{bmatrix} 3&4&5 \end{bmatrix} \end{pmatrix} - \begin{Bmatrix} \begin{pmatrix} \begin{bmatrix} 4&4&4 \end{bmatrix} - \begin{bmatrix} 3&4&5 \end{bmatrix} \end{pmatrix} \cdot \begin{bmatrix} -1&0&0 \end{bmatrix} \end{Bmatrix} \begin{bmatrix} -1&0&0 \end{bmatrix} \end{vmatrix} \\ 
 & = \begin{vmatrix} \begin{bmatrix} 1&0&-1 \end{bmatrix} - \begin{Bmatrix} \begin{bmatrix} 1&0&-1 \end{bmatrix} \cdot \begin{bmatrix} -1&0&0 \end{bmatrix} \end{Bmatrix} \begin{bmatrix} -1&0&0 \end{bmatrix} \end{vmatrix} \\ 
 & = \begin{vmatrix} \begin{bmatrix} 1&0&-1 \end{bmatrix} - \begin{Bmatrix} -1 \end{Bmatrix} \begin{bmatrix} -1&0&0 \end{bmatrix} \end{vmatrix} \\ 
 & = \begin{vmatrix} \begin{bmatrix} 1&0&-1 \end{bmatrix} - \begin{bmatrix} 1&0&0 \end{bmatrix} \end{vmatrix} \\ 
 & = \begin{vmatrix} \begin{bmatrix} 0&0&-1 \end{bmatrix} \end{vmatrix} \\ 
 & = \sqrt{0 + 0 + 1} \\ 
 & = 1 
\end{split}$$

In [1]:
#How this would look in code

## Face Zone Example

#### Creating the Zone

Let's look at a face zone example. We will be looking at the zone that is created by face $\mathbf{f_1}$. We will call the constraint matrix $\mathbf{A}$ and bounds $\mathbf{a}$ for this example $\mathbf{R}$ and $\mathbf{r}$ respectively. Let's first calculate $\mathbf{R}$. The first normal in $\mathbf{R}$ is the negative of the normal of the face, and for this example, that is $- \mathbf{f_1}$. The other normals in $\mathbf{R}$ are calculated by taking the cross product between the corresponding edge vectors and the normal of the face. For this example, the corresponding edge vectors for $\mathbf{f_1}$ are $\mathbf{e_1}$, $\mathbf{e_2}$, $\mathbf{e_4}$, and $\mathbf{e_6}$. By looking at what collumn $\mathbf{f_1}$ is in in $\mathbf{M_{face}}$, we know we need to multiply the cross product between $\mathbf{e_2}$ and $\mathbf{e_6}$ by a negative one so that the directions of all normals in $\mathbf{R}$ point outwards from the zone. For this example, matrix $\mathbf{R}$ is defined as:

$$\mathbf{R} = \begin{bmatrix*}[r] - \mathbf{f_1} \\ \mathbf{e_1} \times \mathbf{f_1} \\ - \mathbf{e_2} \times \mathbf{f_1} \\ \mathbf{e_4} \times \mathbf{f_1} \\ - \mathbf{e_6} \times \mathbf{f_1} \end{bmatrix*} = \begin{bmatrix*}[r] - \begin{bmatrix*}[r]0&0&4 \end{bmatrix*} \\ \begin{bmatrix*}[r]-2&0&0 \end{bmatrix*} \times [0&0&4] \\ - \begin{bmatrix*}[r]0&-2&0 \end{bmatrix*} \times [0&0&4] \\ \begin{bmatrix*}[r]0&-2&0 \end{bmatrix*} \times [0&0&4] \\ - \begin{bmatrix*}[r]-2&0&0 \end{bmatrix*} \times [0&0&4] \end{bmatrix*} = \begin{bmatrix*}[r] 0&0&-4 \\ 0&8&0 \\ 8&0&0 \\ -8&0&0 \\ 0&-8&0 \end{bmatrix*} $$

Now that we have solved for $\mathbf{R}$, we can find the bounds $\mathbf{r}$. We compute $\mathbf{r}$ by taking the dot product between the normals in $\mathbf{R}$ and points that lie on the boundary planes. To do this, let's split up $\mathbf{R}$ into its planar boundary components:

$$\mathbf{R} = \begin{bmatrix} \mathbf{R_1} \\ \mathbf{R_2} \\ \mathbf{R_3} \\ \mathbf{R_4} \\ \mathbf{R_5} \end{bmatrix} = \begin{bmatrix*}[r] 0&0&-4 \\ 0&8&0 \\ 8&0&0 \\ -8&0&0 \\ 0&-8&0 \end{bmatrix*}$$

We use the vertices that correspond with the face for the points that lie on the boundary planes. For this example, the corresponding vertices are $\mathbf{v_1}$, $\mathbf{v_2}$, $\mathbf{v_3}$, and $\mathbf{v_5}$. For $\mathbf{R_1}$ any associated vertex works since they all lie in that boundary plane (Note: In the code, the centroid of the face is used for $\mathbf{R_1}$). For $\mathbf{R_2}$, either vertex $\mathbf{v_1}$ or $\mathbf{v_2}$ works. For $\mathbf{R_3}$, either vertex $\mathbf{v_1}$ or $\mathbf{v_3}$ works. For $\mathbf{R_4}$, either vertex $\mathbf{v_2}$ or $\mathbf{v_5}$ works. For $\mathbf{R_5}$, either vertex $\mathbf{v_3}$ or $\mathbf{v_5}$ works. We will define the bounds $\mathbf{r}$ as:

$$\mathbf{r} = \begin{bmatrix} \mathbf{R_1} \cdot \begin{pmatrix} \mathbf{v_1} + \mathbf{s_{position}} \end{pmatrix} \\ \mathbf{R_2} \cdot \begin{pmatrix} \mathbf{v_1} + \mathbf{s_{position}} \end{pmatrix} \\ \mathbf{R_3} \cdot \begin{pmatrix} \mathbf{v_3} + \mathbf{s_{position}} \end{pmatrix} \\ \mathbf{R_4} \cdot \begin{pmatrix} \mathbf{v_2} + \mathbf{s_{position}} \end{pmatrix} \\ \mathbf{R_5} \cdot \begin{pmatrix} \mathbf{v_5} + \mathbf{s_{position}} \end{pmatrix} \end{bmatrix} $$

<center><img src="Image/face_zone.png" alt="face_zone" width="800"/></center>

Edge zones + dist calc


Face zones + dist calc


Image bounds



